# Column Generation Method to Cutting Stock Problem

In the column generation method only a subset of the variables is used initially. The method sequentially addes columns, using information given by the dual variables for finding the approriate variable to add.

The problem being solved is split into two problems: the **master problem** and the **sub-problem**. 

The **master problem** is the original column-wise formulation of the problem with only a subset of variables being considered. 

The **sub-problem** is a new problem created to identify a new promising variable. 

The *objective function* of the sub-problem is the *reduced cost of the new variable* with respect to the current dual variables, and the constraints require that the variable obeys the naturally occurring constraints.

In [1]:
import gurobipy as grb
from gurobipy import GRB
import numpy as np

The *cutting stock problem* is to decide how to cut a total number of ordered width at $d$ least $b_d$ times, from all the available cutting patterns, so that the total number of base rolls used is minimized. Let $a_{dj}$  represents the number of times the width of order *d* is cut out in the *j*-th cutting pattern and *J* the set of current cutting patters. Then the *master program* can be described as follows:

$
\min  \sum_{j \in J} x_{j}\\
s.t.:  \sum_{j \in J} a_{dj} x_{j} \geq b_{d} \qquad \forall d = 1, \dots, m \\
     x_{j} \geq 0 \qquad \forall j \in J \\
     x_{j} \in \mathbb{Z} \qquad \forall j \in J
$

In [2]:
def master_problem(column, type_program):
    m = grb.Model("Master")
    x = m.addMVar(shape=column.shape[1], lb=0, vtype=type_program)
    
    m.addConstr(lhs=column @ x >= d)
    m.setObjective(x.sum(), GRB.MINIMIZE)
    m.optimize()

    if type_program == GRB.CONTINUOUS:
        return np.array(m.getAttr('Pi', m.getConstrs()))
    else:
        return m.objVal, np.array(m.getAttr('X'))

In [3]:
def lp_master_problem(column):
    return master_problem(column, GRB.CONTINUOUS)

def ip_master_problem(column):
    return master_problem(column, GRB.INTEGER)

Consider a linear relaxation of the master program, and the optimal dual variable vector $\lambda$. Using $\lambda$ assigned to each width $i$, the objctive of the sub-problem is to find a feasible pattern that maximizes the value of the selected widths. Let $y_i$ represents a valid cutting patter. Then the *sub-problem* can be described as follows:

$
\min  \sum_{i = 1}^{m} \lambda_{i} y_{i}\\
s.t.:  \sum_{i = 1}^{m} w_{i} y_{i} \leq W \\
     y_{i} \geq 0 \qquad \forall i = 1, \dots, m \\
     y_{i} \in \mathbb{Z} \qquad \forall i = 1, \dots, m 
$

In [4]:
def subproblem(dual_values):
    m = grb.Model()
    x = m.addMVar(shape=dual_values.shape[0], lb=0, vtype=GRB.INTEGER)
    
    m.addConstr(lhs=w @ x <= W)
    m.setObjective(1 - dual_values @ x, GRB.MINIMIZE)
    m.optimize()

    flag_new_column = m.objVal < 0
    if flag_new_column:
        new_column = m.getAttr('X')
    else:
        new_column = None
    return flag_new_column, new_column

The algorithm is simple:

- First, we solve a linear relaxation of the master problem;
- After, we use the dual information to generate the sub-problem and solve;
- If the optimum of the subproblem is less than 1 then the reduced costs have become all non-negative, and no more patterns are generated. Otherwise, we use the optimum $y$ to generate a new cutting pattern.

Horewer, the algorithm needs of a initial valid cutting patters. We use the following formula to generate the inital cutting patters:

$
\lfloor W/w_i \rfloor
$

In [5]:
def initial_solution(W, w):
    return np.diag(np.floor(W / w))

### Execution

In [6]:
W = np.array(17)
w = np.array([3, 6, 7, 8])
d = np.array([25, 20, 18,10])
initial_pattern = initial_solution(W, w)

In [10]:
flag_new_cut_pattern = True
new_cut_pattern = None
cut_pattern = initial_pattern

while flag_new_cut_pattern:
    if new_cut_pattern:
        cut_pattern = np.column_stack((cut_pattern, new_cut_pattern))
        
    dual_values = lp_master_problem(cut_pattern)
    flag_new_cut_pattern, new_cut_pattern = subproblem(dual_values)


minimal_stock, optimal_number = ip_master_problem(cut_pattern)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xf316e12f
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.900000000e+01
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x9b7c1742
Variable types: 0 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e+00, 8e+0

Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 9 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)

Root relaxation: objective 2.420000e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   24.20000    0    1   29.00000   24.20000  16.6%     -    0s
H    0     0                      25.0000000   24.20000  3.20%     -    0s
     0     0   24.20000    0    1   25.00000   24.20000  3.20%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 25 29 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.500000000000e+01, best bound 2.500000000000e+01, gap 0.0000%


In [12]:
print('************************************************')
print('                 PARAMETERS')
print('************************************************')
print(f'W: {W}')
print(f'w_i: {w}')
print(f'd_i: {d}')
print()
print('************************************************')
print('                  RESULT')
print('************************************************')
print(f'Min Stock: {minimal_stock}')
print(f'Cutting Pattern: \n {cut_pattern}')
print(f'Optimal Number: {optimal_number}')

************************************************
                 PARAMETERS
************************************************
W: 17
w_i: [3 6 7 8]
d_i: [25 20 18 10]

************************************************
                  RESULT
************************************************
Min Stock: 25.0
Cutting Pattern: 
 [[ 5.  0.  0.  0.  1.  1.  3.  1.]
 [ 0.  2.  0.  0.  2. -0. -0.  1.]
 [ 0.  0.  2.  0. -0.  2.  0. -0.]
 [ 0.  0.  0.  2. -0. -0.  1.  1.]]
Optimal Number: [ 1. -0. -0. -0.  5.  9. -0. 10.]
